In [20]:
import pandas as pd
import random
from sklearn.utils import shuffle


In [26]:
#I needed some data processing
female_names= list()
male_names = list()

with open('datasets/names/female.txt') as f:
    female_names = f.readlines()
    female_names = [x.strip() for x in female_names] 
    
with open('datasets/names/male.txt') as m:
    male_names = m.readlines()
    male_names = [x.strip() for x in male_names] 

female_tuples = list()
male_tuples = list()
for i in female_names:
    female_tuples.append((i,'Female'))

for m in male_names:
    male_tuples.append((m,'Male'))

raw_data = female_tuples+male_tuples
names_data = pd.DataFrame(raw_data, columns =['Name', 'Gender']) 
names_data = shuffle(df)
names_data.reset_index(inplace=True, drop=True) 
names_data.head()


,Name,Gender
0,Peyton,Male
1,Kippy,Female
2,Zeb,Male
3,Tiertza,Female
4,Eilis,Female


In [27]:
len(names_data)

7944

In [28]:
len(names_data['Name'].unique())

7578

In [30]:
#clean data (eliminate duplicates)
#and choose randomly a gender for a name that s both for female and male
#example Farah can be female or male we choose randomly its gender association

names_data = names_data.drop_duplicates(subset=['Name'],
                                       keep=random.choice(['first','last']))

In [31]:
len(names_data)

7578

In [32]:
names_data.head()

,Name,Gender
0,Peyton,Male
1,Kippy,Female
2,Zeb,Male
3,Tiertza,Female
4,Eilis,Female


In [33]:
from sklearn import preprocessing

In [34]:
le = preprocessing.LabelEncoder()
names_data['Gender'] = le.fit_transform(names_data['Gender'])
names_data.head()

,Name,Gender
0,Peyton,1
1,Kippy,0
2,Zeb,1
3,Tiertza,0
4,Eilis,0


In [49]:
genders = ['Female','Male']

In [50]:
import string
all_letters = string.ascii_letters+ ".,;'"
all_letters

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,;'"

In [51]:
n_letters = len(all_letters)
n_letters

56

In [52]:
#one hot encoding
import torch

def name_to_tensor(name):
    name_in_tensor = torch.zeros(len(name),1,n_letters)
    for i,letter in enumerate(name):
        name_in_tensor[i][0][all_letters.find(letter)]=1
    
    return name_in_tensor

In [71]:
name_to_tensor('s')

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.]]])

In [72]:
name_to_tensor('seif')

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0

In [73]:
#building the RNN
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size,hidden_size,output_size):
        super(RNN,self).__init__()
        
        self.hidden_size=hidden_size
        
        self.i2h = nn.Linear(input_size+hidden_size,hidden_size)
        self.i2o =nn.Linear(input_size+hidden_size,output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, hidden):
        combined = torch.cat((input,hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output,hidden
    
    def initHidden(self):
        return torch.zeros(1,self.hidden_size) #use to initialize hidden state

In [74]:
#initialization
n_hidden = 128
n_genders= len(genders)

rnn = RNN(n_letters,n_hidden,output_size= n_genders)

In [75]:
iterations = 100000

criterion = nn.NLLLoss()

learning_rate = 0.005

In [76]:
def output_to_gender(output):
    
    top_n,top_index = output.topk(1)
    pred_i = top_index[0].item()
    pred = genders[pred_i]
    return pred

In [77]:
#train

for iteration in range (1, iterations+1):
    
    i = random.randint(0,len(names_data)-1)
    
    name = names_data.iloc[i][0]
    name_in_tensor = name_to_tensor(name) #encode name one hot form
    
    gender = names_data.iloc[i][1]
    gender_in_tensor = torch.LongTensor([gender]) #setup the gender associated 
    #...with the name as a tensor
    
    hidden = rnn.initHidden() #initialize first hidden layer
    rnn.zero_grad() #zero the gradients
    
    for i in range(name_in_tensor.size()[0]):
        output,hidden = rnn(name_in_tensor[i],hidden)
        
    loss = criterion(output,gender_in_tensor)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
    
    if iteration % 2000 == 0:
        pred = output_to_gender(output)
        correct ='✓' if pred == genders[gender] else 'X {%s}' % genders[gender]
        print('iters- %d %d%% (%s) Name - %s Gender - %s %s' % \
             (iteration,iteration/iterations*100,loss.item(),name,pred,correct))


iters- 2000 2% (0.19884511828422546) Name - Nerta Gender - Female ✓
iters- 4000 4% (0.1525212973356247) Name - Tedra Gender - Female ✓
iters- 6000 6% (0.10378465056419373) Name - Rebeca Gender - Female ✓
iters- 8000 8% (0.7178431153297424) Name - Dynah Gender - Male X {Female}
iters- 10000 10% (0.053023405373096466) Name - Arlana Gender - Female ✓
iters- 12000 12% (0.0400589182972908) Name - Alberta Gender - Female ✓
iters- 14000 14% (1.0757181644439697) Name - Rianon Gender - Male X {Female}
iters- 16000 16% (0.43146172165870667) Name - Olympie Gender - Female ✓
iters- 18000 18% (0.017173511907458305) Name - Carma Gender - Female ✓
iters- 20000 20% (0.4761023223400116) Name - Alleen Gender - Female ✓
iters- 22000 22% (0.21133175492286682) Name - Shaylyn Gender - Female ✓
iters- 24000 24% (0.1772412210702896) Name - Tremain Gender - Male ✓
iters- 26000 26% (1.2395676374435425) Name - Scottie Gender - Female X {Male}
iters- 28000 28% (0.021834347397089005) Name - Lorianne Gender - Femal

In [84]:
#will generate confusion matrix
n_confusion = 10000

prediction = []
actual = []

for _ in range (n_confusion):
    i = random.randint(0,len(names_data)-1)
    
    name = names_data.iloc[i][0]
    name_in_tensor = name_to_tensor(name)
    
    gender_idx = names_data.iloc[i][1]
    gender_in_tensor = torch.LongTensor([gender_idx])
    
    hidden = rnn.initHidden()
    
    for j in range(name_in_tensor.size()[0]):
        output, hidden = rnn(name_in_tensor[j],hidden)
    pred = output_to_gender(output)
    
    prediction.append(pred)
    actual.append(genders[gender_idx])

In [94]:
import numpy as np

np_prediction =np.array(prediction)
np_actual = np.array(actual)

In [97]:
T = 0
total =0

for x in range(len(np_prediction)):
    if np_prediction[x] == np_actual[x]:
        T += 1
    total += 1

print('Accuracy = %.4f %%'% ((T/total)*100 ) )
print('right values : %d' % T)
        

Accuracy = 83.4200 %
right values : 8342
